In [ ]:
!pip install contractions

     |████████████████████████████████| 327kB 4.3MB/s 
     |████████████████████████████████| 286kB 6.1MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85400 sha256=546488852274c3f16b5d7ab11cb0d4f82e09761609bad70806cebf8ddaf7b471
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [ ]:
#imports
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import contractions
from tqdm import tqdm
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
import json
from sklearn.metrics import f1_score

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data = pd.read_csv('train.tsv',sep='\t',error_bad_lines=False)

b'Skipping line 93: expected 5 fields, saw 6\nSkipping line 590: expected 5 fields, saw 6\nSkipping line 778: expected 5 fields, saw 6\nSkipping line 790: expected 5 fields, saw 6\nSkipping line 882: expected 5 fields, saw 6\nSkipping line 1296: expected 5 fields, saw 6\nSkipping line 1324: expected 5 fields, saw 6\nSkipping line 1640: expected 5 fields, saw 6\nSkipping line 1759: expected 5 fields, saw 6\nSkipping line 1808: expected 5 fields, saw 6\nSkipping line 2010: expected 5 fields, saw 6\nSkipping line 2254: expected 5 fields, saw 6\nSkipping line 2268: expected 5 fields, saw 6\nSkipping line 2527: expected 5 fields, saw 6\nSkipping line 2602: expected 5 fields, saw 6\nSkipping line 2659: expected 5 fields, saw 6\nSkipping line 2929: expected 5 fields, saw 6\nSkipping line 3032: expected 5 fields, saw 6\nSkipping line 3144: expected 5 fields, saw 6\nSkipping line 3322: expected 5 fields, saw 6\nSkipping line 3563: expected 5 fields, saw 6\n'


In [ ]:
data.head()

,Quality,#1 ID,#2 ID,#1 String,#2 String
0,1,702876,702977,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr..."
1,0,2108705,2108831,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ..."
3,0,3344667,3344648,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set..."
4,1,1236820,1236712,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...


In [ ]:
data.shape

(3549, 5)

In [ ]:
data.isnull().sum()

Quality       0
#1 ID         0
#2 ID         0
#1 String     0
#2 String    20
dtype: int64

In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

Quality      0
#1 ID        0
#2 ID        0
#1 String    0
#2 String    0
dtype: int64

In [ ]:
data.columns

Index(['Quality', '#1 ID', '#2 ID', '#1 String', '#2 String'], dtype='object')

In [ ]:
data['Quality'].value_counts()

1    2379
0    1150
Name: Quality, dtype: int64

In [ ]:
y_label=data['Quality']

In [ ]:
def expand_contractions(text):
  text = [contractions.fix(word) for word in text.split()]
  text = ' '.join(text)
  return text

In [ ]:
def pre_process_data(text):
  ps = PorterStemmer()
  text = expand_contractions(text)
  text = re.sub(r"[^A-Za-z0-9]", " ", text)
  text = text.lower()
  text = text.split()
  text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
  text = ' '.join(text)
  return text



In [ ]:
data.iloc[0,3]

'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'

In [ ]:
data.iloc[0,4]

'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'

In [ ]:
corpus_msg_1 = []
corpus_msg_2 = []
for i in range(data.shape[0]):
  processed_text = pre_process_data(data.iloc[i,3])
  corpus_msg_1.append(processed_text)
  processed_text = pre_process_data(data.iloc[i,4])
  corpus_msg_2.append(processed_text)
  


In [ ]:
print(corpus_msg_1)

['amrozi accus brother call wit deliber distort evid', 'yucaipa own dominick sell chain safeway 1998 2 5 billion', 'publish advertis internet june 10 offer cargo sale ad', 'around 0335 gmt tab share 19 cent 4 4 4 56 earlier set record high 4 57', 'stock rose 2 11 11 percent close friday 21 51 new york stock exchang', 'revenu first quarter year drop 15 percent period year earlier', 'nasdaq weekli gain 17 27 1 2 percent close 1 520 15 friday', 'dvd cca appeal state suprem court', 'compar 35 18 million 24 cent per share year ago period', 'share genentech much larger compani sever product market rose 2 percent', 'legisl make harder consum eras debt bankruptci court overwhelm hous approv march', 'nasdaq composit index increas 10 73 0 7 percent 1 514 77', 'ad group perform would improv second half year beyond', 'told sun newspap mr hussein daughter british school hospit mind decid ask asylum', 'gyorgi heizler head local disast unit said coach carri 38 passeng', 'rudder recent senior vice pre

In [ ]:
print(corpus_msg_2)

['refer wit amrozi accus brother deliber distort evid', 'yucaipa bought dominick 1995 693 million sold safeway 1 8 billion 1998', 'june 10 ship owner publish advertis internet offer explos sale', 'tab share jump 20 cent 4 6 set record close high 4 57', 'pg e corp share jump 1 63 8 percent 21 03 new york stock exchang friday', 'scandal hang stewart compani revenu first quarter year drop 15 percent period year earlier', 'tech lace nasdaq composit ixic ralli 30 46 point 2 04 percent 1 520 15', 'dvd cca appeal decis suprem court', 'earn affect non recur 8 million tax benefit year ago period', 'share xoma fell 16 percent earli trade share genentech much larger compani sever product market 2 percent', 'legisl make harder consum eras debt bankruptci court speedi hous approv march endors white hous', 'nasdaq composit index full technolog stock late around 18 point', 'de sole said result statement group perform would improv second half year beyond', 'saddam daughter british school hospit mind d

In [ ]:
count = Counter()
for i in range(len(corpus_msg_1)):
  for word in corpus_msg_1[i].split():
    count[word] +=1
  for word in corpus_msg_2[2].split():
    count[word] +=1

In [ ]:
print(len(count))

7802


In [ ]:
num_words = len(count)
max_length = 50

In [ ]:
train_size = int(data.shape[0]*.80)

In [ ]:
x_train_1 = corpus_msg_1[:train_size]
x_train_2 = corpus_msg_2[:train_size]
x_val_1 = corpus_msg_1[train_size:]
x_val_2 = corpus_msg_2[train_size:]

y_train = y_label[:train_size]
y_val = y_label[train_size:]

In [ ]:
x_train_1+x_train_2

['amrozi accus brother call wit deliber distort evid',
 'yucaipa own dominick sell chain safeway 1998 2 5 billion',
 'publish advertis internet june 10 offer cargo sale ad',
 'around 0335 gmt tab share 19 cent 4 4 4 56 earlier set record high 4 57',
 'stock rose 2 11 11 percent close friday 21 51 new york stock exchang',
 'revenu first quarter year drop 15 percent period year earlier',
 'nasdaq weekli gain 17 27 1 2 percent close 1 520 15 friday',
 'dvd cca appeal state suprem court',
 'compar 35 18 million 24 cent per share year ago period',
 'share genentech much larger compani sever product market rose 2 percent',
 'legisl make harder consum eras debt bankruptci court overwhelm hous approv march',
 'nasdaq composit index increas 10 73 0 7 percent 1 514 77',
 'ad group perform would improv second half year beyond',
 'told sun newspap mr hussein daughter british school hospit mind decid ask asylum',
 'gyorgi heizler head local disast unit said coach carri 38 passeng',
 'rudder recent 

In [ ]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(x_train_1+x_train_2)

In [ ]:
x_train_1[0]

'amrozi accus brother call wit deliber distort evid'

In [ ]:
x_train_sq_1 = tokenizer.texts_to_sequences(x_train_1)
x_train_sq_2 = tokenizer.texts_to_sequences(x_train_2)

In [ ]:
x_train_sq_1[0]

[1123, 294, 927, 47, 710, 3280, 3857, 307]

In [ ]:
x_val_sq_1 = tokenizer.texts_to_sequences(x_val_1)
x_val_sq_2 = tokenizer.texts_to_sequences(x_val_2)

In [ ]:
x_val_sq_1[0]

[86, 1, 46, 7, 1119, 587, 729, 281, 2286, 339, 132, 266, 425, 563, 292]

In [ ]:
train_padded_1 = pad_sequences(x_train_sq_1, maxlen=max_length, padding="post", truncating="post")
train_padded_2 = pad_sequences(x_train_sq_2, maxlen=max_length, padding="post", truncating="post")
val_padded_1 = pad_sequences(x_val_sq_1, maxlen=max_length, padding="post", truncating="post")
val_padded_2 = pad_sequences(x_val_sq_2, maxlen=max_length, padding="post", truncating="post")

In [ ]:
train_padded_1[0]

array([1123,  294,  927,   47,  710, 3280, 3857,  307,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
train_padded_2[0]

array([1423,  710, 1123,  294,  927, 3280, 3857,  307,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
val_padded_1[0]

array([  86,    1,   46,    7, 1119,  587,  729,  281, 2286,  339,  132,
        266,  425,  563,  292,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
val_padded_2[0]

array([  86,    1,   46,    7, 1119,  587,  729,  281,  339,  132,  229,
        266,  425,  563,  711,  292, 1342, 1931,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
print(train_padded_1.shape)
print(train_padded_2.shape)
print(val_padded_1.shape)
print(val_padded_2.shape)

(2823, 50)
(2823, 50)
(706, 50)
(706, 50)


Model Creation

In [ ]:
embedding_layer = Embedding(num_words,150,input_length=max_length)


seq_1_input = Input(shape=(max_length,))
embedded_seq_1 = embedding_layer(seq_1_input)
x1 = LSTM(64,dropout=0.2,recurrent_dropout=0.2)(embedded_seq_1)

seq_2_input = Input(shape=(max_length,))
embedded_seq_2 = embedding_layer(seq_2_input)
x2 = LSTM(64,dropout=0.2,recurrent_dropout=0.2)(embedded_seq_2)

merged = concatenate([x1, x2])
merged = Dropout(0.3)(merged)
#merged = BatchNormalization()(merged)

merged = Dense(24, activation='relu')(merged)
merged = Dropout(0.2)(merged)
#merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[seq_1_input, seq_2_input],outputs=preds)
#opt = SGD(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 150)      1170300     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 64)           55040       embedding_1[0][0]          

In [ ]:
filepath='/content/drive/MyDrive/Saved Models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit([train_padded_1, train_padded_2], y_train,validation_data=([val_padded_1, val_padded_2],y_val),epochs=5,batch_size=24,callbacks=callbacks_list)

Epoch 1/5
118/118 [==============================] - 41s 345ms/step - loss: 0.6360 - accuracy: 0.6695 - val_loss: 0.6218 - val_accuracy: 0.6926

Epoch 00001: val_accuracy did not improve from 0.69263
Epoch 2/5
118/118 [==============================] - 41s 346ms/step - loss: 0.6367 - accuracy: 0.6699 - val_loss: 0.6174 - val_accuracy: 0.6926

Epoch 00002: val_accuracy did not improve from 0.69263
Epoch 3/5
118/118 [==============================] - 41s 346ms/step - loss: 0.6386 - accuracy: 0.6699 - val_loss: 0.6174 - val_accuracy: 0.6926

Epoch 00003: val_accuracy did not improve from 0.69263
Epoch 4/5
118/118 [==============================] - 40s 343ms/step - loss: 0.6361 - accuracy: 0.6702 - val_loss: 0.6215 - val_accuracy: 0.6926

Epoch 00004: val_accuracy did not improve from 0.69263
Epoch 5/5
118/118 [==============================] - 41s 346ms/step - loss: 0.6374 - accuracy: 0.6702 - val_loss: 0.6195 - val_accuracy: 0.6926

Epoch 00005: val_accuracy did not improve from 0.69263


In [ ]:
pre= model.predict([train_padded_2[0].reshape(-1,50),train_padded_1[0].reshape(-1,50)])
print(pre)


[[0.704882]]


In [ ]:
train_padded_2[0].reshape(-1,50)

In [ ]:
model.save('/content/drive/MyDrive/Saved Models/final_nlp.model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Saved Models/final_nlp.model/assets


In [ ]:
tokenizer_details = tokenizer.to_json()

In [ ]:
tokenizer_details

'{"class_name": "Tokenizer", "config": {"num_words": 7802, "filters": "!\\"#$%&()*+,-./:;<=>?@[\\\\]^_`{|}~\\t\\n", "lower": true, "split": " ", "char_level": false, "oov_token": null, "document_count": 5646, "word_counts": "{\\"amrozi\\": 13, \\"accus\\": 44, \\"brother\\": 16, \\"call\\": 124, \\"wit\\": 21, \\"deliber\\": 3, \\"distort\\": 2, \\"evid\\": 43, \\"yucaipa\\": 2, \\"own\\": 24, \\"dominick\\": 2, \\"sell\\": 23, \\"chain\\": 11, \\"safeway\\": 4, \\"1998\\": 23, \\"2\\": 235, \\"5\\": 141, \\"billion\\": 141, \\"publish\\": 27, \\"advertis\\": 20, \\"internet\\": 30, \\"june\\": 70, \\"10\\": 117, \\"offer\\": 41, \\"cargo\\": 8, \\"sale\\": 102, \\"ad\\": 71, \\"around\\": 55, \\"0335\\": 1, \\"gmt\\": 10, \\"tab\\": 6, \\"share\\": 282, \\"19\\": 46, \\"cent\\": 219, \\"4\\": 232, \\"56\\": 13, \\"earlier\\": 71, \\"set\\": 27, \\"record\\": 48, \\"high\\": 75, \\"57\\": 15, \\"stock\\": 139, \\"rose\\": 104, \\"11\\": 58, \\"percent\\": 622, \\"close\\": 112, \\"frid

In [ ]:
out_file = open("/content/drive/MyDrive/Saved Models/tokenizer_details.json", "w")
json.dump(tokenizer_details, out_file)
out_file.close()

In [ ]:
data_test = pd.read_csv('test.tsv',sep='\t',error_bad_lines=False)

b'Skipping line 34: expected 5 fields, saw 6\nSkipping line 121: expected 5 fields, saw 6\nSkipping line 211: expected 5 fields, saw 6\nSkipping line 263: expected 5 fields, saw 6\nSkipping line 345: expected 5 fields, saw 6\nSkipping line 696: expected 5 fields, saw 6\nSkipping line 733: expected 5 fields, saw 6\nSkipping line 847: expected 5 fields, saw 6\nSkipping line 1392: expected 5 fields, saw 6\nSkipping line 1467: expected 5 fields, saw 6\nSkipping line 1551: expected 5 fields, saw 6\n'


In [ ]:
dummy = pd.read_csv('msr_paraphrase_test.csv',usecols=['Quality','#1 ID','#2 ID'],error_bad_lines=False)

In [ ]:
dummy.head()

,Quality,#1 ID,#2 ID
0,1,1089874,1089925
1,1,3019446,3019327
2,1,1945605,1945824
3,0,1430402,1430329
4,0,3354381,3354396


In [ ]:
data_test.head()

,index,#1 ID,#2 ID,#1 String,#2 String
0,0,1089874,1089925,"PCCW 's chief operating officer , Mike Butcher...",Current Chief Operating Officer Mike Butcher a...
1,1,3019446,3019327,The world 's two largest automakers said their...,Domestic sales at both GM and No. 2 Ford Motor...
2,2,1945605,1945824,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...
3,3,1430402,1430329,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...
4,4,3354381,3354396,The company didn 't detail the costs of the re...,But company officials expect the costs of the ...


In [ ]:
data_test.shape

(1639, 5)

In [ ]:
dummy.shape

(1725, 3)

In [ ]:
data_test = pd.merge(data_test,dummy,on=['#1 ID','#2 ID'])

In [ ]:
data_test.shape

(1639, 6)

In [ ]:
data_test.head()

,index,#1 ID,#2 ID,#1 String,#2 String,Quality
0,0,1089874,1089925,"PCCW 's chief operating officer , Mike Butcher...",Current Chief Operating Officer Mike Butcher a...,1
1,1,3019446,3019327,The world 's two largest automakers said their...,Domestic sales at both GM and No. 2 Ford Motor...,1
2,2,1945605,1945824,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...,1
3,3,1430402,1430329,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...,0
4,4,3354381,3354396,The company didn 't detail the costs of the re...,But company officials expect the costs of the ...,0


In [ ]:
data_test.isnull().sum()

index        0
#1 ID        0
#2 ID        0
#1 String    0
#2 String    9
Quality      0
dtype: int64

In [ ]:
data_test = data_test.dropna()

In [ ]:
data_test.isnull().sum()

index        0
#1 ID        0
#2 ID        0
#1 String    0
#2 String    0
Quality      0
dtype: int64

In [ ]:
data_test.shape

(1630, 6)

In [ ]:
y_test=data_test['Quality']

In [ ]:
corpus_test_msg_1 = []
corpus_test_msg_2 = []
for i in range(data_test.shape[0]):
  processed_text = pre_process_data(data_test.iloc[i,3])
  corpus_test_msg_1.append(processed_text)
  processed_text = pre_process_data(data_test.iloc[i,4])
  corpus_test_msg_2.append(processed_text)
  


In [ ]:
corpus_test_msg_1

['pccw chief oper offic mike butcher alex arena chief financi offic report directli mr',
 'world two largest automak said sale declin predict last month late summer sale frenzi caus industri backlash expect',
 'accord feder center diseas control prevent news web site 19 report case measl unit state 2002',
 'tropic storm rapidli develop gulf mexico sunday expect hit somewher along texa louisiana coast monday night',
 'compani detail cost replac repair',
 'settl compani would also assign possibl claim underwrit investor plaintiff ad',
 'air commodor quaif said hornet remain three minut alert throughout oper',
 'washington counti man may counti first human case west nile viru health depart said friday',
 'moseley senior aid deliv summari assess 300 american alli militari offic thursday',
 'broader standard poor 500 index spx 0 46 point lower 0 05 percent 997 02',
 'consum would still get descrambl secur card cabl oper plug set',
 'increas reflect lower credit loss favor interest rate',
 '

In [ ]:
corpus_test_msg_2

['current chief oper offic mike butcher group chief financi offic alex arena report',
 'domest sale gm 2 ford motor co declin predict late summer sale frenzi prompt larger expect industri backlash',
 'center diseas control prevent said 19 report case measl unit state 2002',
 'tropic storm rapidli develop gulf mexico sunday could hurrican forc wind hit land somewher along louisiana coast monday night',
 'compani offici expect cost replac work run million dollar',
 'agreement settl compani also assign potenti claim underwrit investor ad',
 'air commodor john quaif said secur oper unpreced',
 'counti first human case west nile year confirm health offici sept 8',
 'gener moseley senior aid present assess intern brief american alli militari offic nelli air forc base nevada thursday',
 'technolog lace nasdaq composit index ixic 7 42 point 0 45 percent 1 653 44',
 'watch pay televis consum would insert set secur card provid cabl servic',
 'gain came result fewer credit loss lower interest rat

In [ ]:
x_test_sq_1 = tokenizer.texts_to_sequences(corpus_test_msg_1)
x_test_sq_2 = tokenizer.texts_to_sequences(corpus_test_msg_2)

In [ ]:
test_padded_1 = pad_sequences(x_test_sq_1, maxlen=max_length, padding="post", truncating="post")
test_padded_2 = pad_sequences(x_test_sq_2, maxlen=max_length, padding="post", truncating="post")

In [ ]:
print(test_padded_1.shape)
print(test_padded_2.shape)

(1630, 50)
(1630, 50)


In [ ]:
pre= model.predict([test_padded_1.reshape(-1,50),test_padded_2.reshape(-1,50)])

In [ ]:
print(pre)
print(pre.shape)

[[0.7048827 ]
 [0.70488375]
 [0.704882  ]
 ...
 [0.70488214]
 [0.7048925 ]
 [0.70488393]]
(1630, 1)


In [ ]:
y_predicted = []
for i in range(pre.shape[0]):
  if pre[i]>0.5:
    y_predicted.append(1)
  else:
    y_predicted.append(0)

In [ ]:
score = f1_score(y_test, y_predicted)
print('F-Measure: %.3f' % score)

F-Measure: 0.798
